In [10]:
import pandas as pd 

In [11]:
earthquakes = pd.read_csv('earthquake_merged2.csv')
# earthquakes.columns

In [12]:
earthquakes = earthquakes[earthquakes["Year"] > 1999]
# earthquakes

In [13]:
country_counts = earthquakes.groupby("Country Name").size().reset_index(name="Count")
country_counts = country_counts.sort_values(by="Count", ascending=False)
country_counts

,Country Name,Count
17,CHINA,155
46,INDONESIA,140
47,IRAN,86
49,JAPAN,80
103,UNITED STATES,58
...,...,...
38,GULF OF MEXICO,1
51,KAZAKHSTAN,1
50,JORDAN,1
86,SLOVENIA,1


In [14]:
# Group by Country Name to get avg magnitude
avg_magnitude = earthquakes.groupby("Country Name")["Mag"].mean().reset_index(name="Average Magnitude")

# Merge the average magnitude with the country_counts df
country_counts = country_counts.merge(avg_magnitude, on="Country Name", how="left")

# country_counts

In [15]:
import plotly.express as px

fig = px.treemap(
    country_counts,
    path=["Country Name"],
    values="Count",
    title="Earthquake Occurrences by Country",
    hover_data={"Count": True, "Average Magnitude": True}  # Pass 'Average Magnitude' explicitly
)

# custom hover 
fig.update_traces(
    customdata=country_counts[["Average Magnitude"]].values,  # Include custom data for hovertemplate
    hovertemplate="<b>%{label}</b><br>Count: %{value}<br>Average Magnitude: %{customdata[0]:.2f}<extra></extra>"
)

fig.show()


In [16]:
# group by country and year, calculating total counts and sum of damages per year 
bubble_data = earthquakes.groupby(["Country Name", "Year"]).agg(
    Earthquake_Count=("Country Name", "count"),
    Total_Damage_Mil=("Total Damage ($Mil)", "sum")
).reset_index()

# maybe remove the coutnries that dont have totaldamagemil???
fig = px.scatter(
    bubble_data,
    x="Year",
    y="Earthquake_Count",
    size="Total_Damage_Mil",
    color="Country Name",
    title="Earthquake Counts Over Time",
    labels={"Earthquake_Count": "Earthquake Counts", "Total_Damage_Mil": "Total Damage ($Mil)"},
    hover_data={"Total_Damage_Mil": True, "Country Name": True}
)

# improving layout 
fig.update_layout(
    xaxis_title="Year",
    yaxis_title="Earthquake Counts",
    legend_title="Country",
)
fig.show()
